In [160]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [161]:
data = pd.read_csv('../data/fullsample.csv', nrows=1700)
data

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
...,...,...,...,...,...,...,...,...,...,...,...,...
1695,31811982_226,COMPLETED,2021-09-01T00:45:09,2021-09-01T01:58:27,1024Mn,360.55M,02:45:00,01:13:18,1,1,production,0:0
1696,31811982_227,COMPLETED,2021-09-01T00:45:09,2021-09-01T01:58:45,1024Mn,360.49M,02:45:00,01:13:36,1,1,production,0:0
1697,31811982_228,COMPLETED,2021-09-01T00:45:09,2021-09-01T02:03:05,1024Mn,361.14M,02:45:00,01:17:56,1,1,production,0:0
1698,31811982_229,COMPLETED,2021-09-01T00:45:09,2021-09-01T02:03:45,1024Mn,360.99M,02:45:00,01:18:36,1,1,production,0:0


In [163]:
data = data.drop(data[data['END'] == 'Unknown'].index)
data = data[data['EXITCODE'] == '0:0']
data.head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17T07:45:07,2021-09-06T16:17:34,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0


In [165]:
data['END'] = pd.to_datetime(data['END'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1693 entries, 1 to 1699
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   JOBID      1693 non-null   object        
 1   STATE      1693 non-null   object        
 2   BEGIN      1693 non-null   object        
 3   END        1693 non-null   datetime64[ns]
 4   REQMEM     1693 non-null   object        
 5   USEDMEM    1693 non-null   object        
 6   REQTIME    1693 non-null   object        
 7   USEDTIME   1693 non-null   object        
 8   NODES      1693 non-null   int64         
 9   CPUS       1693 non-null   int64         
 10  PARTITION  1693 non-null   object        
 11  EXITCODE   1693 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 171.9+ KB


In [166]:
data = data.sort_values('END')

data['total_every_hour'] = (data
                           .rolling('1h', on = 'END')['STATE']
                           .count())



In [167]:
plt.figure(figsize = (12,6))
data[traffic['END']].set_index('END')['total_every_hour'].plot(color = 'red', label = '30 Day Moving Average')

NameError: name 'traffic' is not defined

<Figure size 864x432 with 0 Axes>

In [130]:
data['BEGIN_DATE'] = pd.to_datetime(data['BEGIN']).dt.date
data['BEGIN_HOUR'] = pd.to_datetime(data['BEGIN']).dt.hour
#data['BEGIN_HOUR'] = data[data['BEGIN_HOUR'].strftime('%H')]

data['END_DATE'] = pd.to_datetime(data['END']).dt.date
data['END_HOUR'] = pd.to_datetime(data['END']).dt.time

data = data[['JOBID', 'STATE', 'BEGIN_DATE', 'BEGIN_HOUR', 'END_DATE', 'END_HOUR', 'PARTITION', 'EXITCODE', 'REQMEM', 'USEDMEM', 'REQTIME', 'USEDTIME', 'NODES', 'CPUS']]

data

,JOBID,STATE,BEGIN_DATE,BEGIN_HOUR,END_DATE,END_HOUR,PARTITION,EXITCODE,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS
1,30853133,COMPLETED,2021-08-06,11,2021-09-05,11:36:32,cgw-platypus,0:0,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1
2,30858137,COMPLETED,2021-08-06,19,2021-09-05,19:04:53,cgw-tbi01,0:0,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32
3,30935078,COMPLETED,2021-08-09,16,2021-09-07,20:52:55,cgw-platypus,0:0,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8
4,31364111_2,COMPLETED,2021-08-17,7,2021-09-10,16:45:24,production,0:0,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1
5,31364111_3,COMPLETED,2021-08-17,7,2021-09-06,16:17:34,production,0:0,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1
6,31364111_4,COMPLETED,2021-08-17,7,2021-09-06,06:25:11,production,0:0,16384Mn,9785.61M,24-09:00:00,19-22:40:04,1,1
7,31364111_5,COMPLETED,2021-08-17,7,2021-09-06,10:05:33,production,0:0,16384Mn,9966.38M,24-09:00:00,20-02:20:26,1,1
8,31364111_6,COMPLETED,2021-08-17,7,2021-09-05,12:53:04,production,0:0,16384Mn,9684.02M,24-09:00:00,19-05:07:57,1,1
9,31364111_9,COMPLETED,2021-08-17,7,2021-09-06,09:10:13,production,0:0,16384Mn,9802.24M,24-09:00:00,20-01:25:06,1,1
